In [1]:
import pandas as pd

from pendulum import now

In [2]:
dataset_name = 'sidra_tab3939_dejetos'
raiz = '../database'

### Bronze

In [60]:
contador = 1

ultimo_ano = now().year - 1
for ano in range(2000, ultimo_ano):
    if contador == 1:
        df_3939 = pd.read_json(f'https://apisidra.ibge.gov.br/values/t/3939/n6/all/v/all/p/{ano}/c79/32793,32794,32796')
        contador += 1
    else:
        df_3939 = pd.concat(
            [df_3939,
             (pd.read_json(f'https://apisidra.ibge.gov.br/values/t/3939/n6/all/v/all/p/{ano}/c79/32793,32794,32796')
              .iloc[1:])],
            ignore_index=True
            )
        contador += 1

df_3939.to_json(f'{raiz}/bronze_{dataset_name}.json')

### Prata

|  CÓDIGO |        CULTURA        |
|---------|-----------------------|
| 32793   | Galináceos - galinhas |
| 327930  | Galináceos - frangos  |
| 32794   |  Suínos - total       |
| 32796   | Galináceos - total    |

In [4]:
df = (pd.read_json(f'{raiz}/bronze_{dataset_name}.json')
 .drop(0, axis = 0)
 .filter(['D1C', 'D3C', 'D4C', 'V'])
 .query('D4C in ("32793", "32794", "32796")')
 .reset_index(drop=True)
 .assign(**{
     'V': lambda df: pd.to_numeric(df['V'], errors='coerce') 
    })
    )

df_frangos = (pd.DataFrame()
             .assign(**{
                 'D1C': df.iloc[1::3, 0].reset_index(drop=True),
                 'D3C': df.iloc[1::3, 1].reset_index(drop=True),
                 'D4C': '327930',
                 'V': df.iloc[2::3, 3].reset_index(drop=True) - df.iloc[::3, 3].reset_index(drop=True)
             })
            )

(pd.concat([df, df_frangos])
 .query('D4C != "32796"')
 .assign(**{
        'uf': lambda df: df['D1C'].str.slice(0,2).astype(int),
        'kg_cabeca': lambda df: pd.to_numeric(df['V'], errors='coerce'),
        'municipio': lambda df: df['D1C'].astype(int),
        'ano': lambda df: df['D3C'].astype(int),
        'cultura': lambda df: df['D4C'].astype(int),
        'fonte': 'Dejetos'
        })
 .dropna(subset='kg_cabeca')
 .convert_dtypes()
 .groupby(['uf', 'municipio', 'ano', 'cultura', 'fonte'])
 .agg({'kg_cabeca':sum})
 .to_parquet(f'{raiz}/prata_{dataset_name}.parquet')
 )

kg_cabeca
uf municipio ano  cultura fonte             
11 1100015   2000 32793   Dejetos      86520
                  32794   Dejetos      22682
                  327930  Dejetos     105060
             2001 32793   Dejetos      51059
                  32794   Dejetos      24497
...                                      ...
53 5300108   2020 32794   Dejetos     186698
                  327930  Dejetos    7763978
             2021 32793   Dejetos    1586344
                  32794   Dejetos     171098
                  327930  Dejetos    7787430

[359605 rows x 1 columns]

### Ouro

Cálculos:

N: ((kg_cabeça * Teor N)/1000) * ((100 - perdas_N)/100)

P, K: (kg_cabeca * Teor N, K)/1000

In [7]:
parametro = {
    32793: {'n': 0.58, 'p': 0.18, 'k': 0.21}, # de Galinhas
    327930: {'n': 0.4, 'p': 0.12, 'k': 0.24}, # de Frangos
    32794: {'n': 14.2, 'p': 2.3, 'k': 6.2}, # Suínos
}

# Perdas por volatilização do N
perdas_N_galinaceos = 0
perdas_N_suinos = 0

In [23]:
df = (pd.read_parquet(f'{raiz}/prata_{dataset_name}.parquet')
 .groupby(['uf', 'ano', 'cultura', 'fonte'])
 .agg({'kg_cabeca':sum})
 .reset_index()
 .assign(
      N = lambda df: df.apply(lambda lin: ((lin['kg_cabeca'] * parametro[lin['cultura']]['n'])/1000) * ((100 - perdas_N_suinos)/100) 
          if lin['cultura'] == 32794 else
          ((lin['kg_cabeca'] * parametro[lin['cultura']]['n'])/1000) * ((100 - perdas_N_galinaceos)/100), axis = 1),
      P= lambda df: df.apply(lambda lin: (lin['kg_cabeca'] * parametro[lin['cultura']]['p'])/1000, axis = 1),
      K= lambda df: df.apply(lambda lin: (lin['kg_cabeca'] * parametro[lin['cultura']]['k'])/1000, axis = 1)
  )
 .drop('kg_cabeca', axis = 1)
 .melt(id_vars=['uf', 'ano', 'cultura', 'fonte'],
       value_vars=['N', 'P', 'K'],
       var_name='elemento',
       value_name='valor')
 .convert_dtypes()
 .groupby(['uf', 'ano', 'fonte', 'elemento'])
 .agg({'valor':sum})
 .reset_index(level='elemento')
 )

df.to_csv(f'{raiz}/ouro_{dataset_name}.csv')
df

elemento       valor
uf ano  fonte                       
11 2000 Dejetos        K  4056.29893
        Dejetos        N   9087.0105
        Dejetos        P  1837.00594
   2001 Dejetos        K  3985.31764
        Dejetos        N  8981.15592
...                  ...         ...
53 2020 Dejetos        N  6557.23678
        Dejetos        P  1609.52434
   2021 Dejetos        K  3262.92304
        Dejetos        N  6464.64312
        Dejetos        P  1613.55892

[1782 rows x 2 columns]